## Data collection & storage...
*Collect updated music track list and lyrics from web...*
    
#### Data Sources ####
*All data has been collected via web-scrapes or developer apis...*
- Spotify Api
- Genius Api
- Genius Website

##### Soon-to-be-Sources #####
- Facebook Api
- Apple Music Api

###### Known Data Source Columns ######
- Spotify Acoustic Features (11 total, ~2 need normalizing)
    - energy
    - liveness
    - tempo
    - speechiness
    - acousticness
    - instrumentalness
    - time_signature
    - danceability
    - duration_ms
    - loudness
    - valence
- Utils for data_mapping (2)
    - utils_spotify_id
    - utils_genius_data

### NOTE: Will need to re-download all song lyrics

### Imports...

#### Tools... 

In [1]:
# TODO: Figure out if this is the complete list of imports needed...

# libraries & tool imports...
import os
import re
import ast
import sys
import json
import pprint
import requests

import numpy as np
import pandas as pd
from os import listdir
from tqdm import tqdm
from os.path import isfile, join
from datetime import datetime
from spotify import spotifyApi
from text_miner import textMiner
from text_miner import geniusApi

In [2]:
# initializing helpers..
helper = spotifyApi()
txtMiner = textMiner()
genius = geniusApi()

In [6]:
# existing lyrics dataset imports...
data_path = "/Users/dayoorigunwa/code_base/music_mapping/data/"
allfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

gScrapeLyrics = pd.read_csv(data_path + "SpotifySongLyrics.csv", encoding="utf-8")
gScrapeLyrics.columns = ["name", "lyrics"]
gApiData = pd.read_csv(data_path + "GeniusAPISearchResults", encoding="utf-8")
# rename cols example -> df2 = df.rename({'a': 'X', 'b': 'Y'}, axis=1)

#### Testing encoded import...

In [7]:
# # explicit utf-8 import...
# utf_scrape_df = pd.read_csv(data_path + "SpotifySongLyrics.csv", encoding='utf-8')
# utf_scrape_df.columns = ['name', 'lyrics']

# # GET unique tracklist
# utf_songs = utf_scrape_df.name.unique()

# # comparing with non-encoded import

# raw_songs = gScrapeLyrics.name.unique()
# diff_songs = [song for song in utf_songs if song not in raw_songs]
# print(len(diff_songs),diff_songs)

# # ...sadness

#### Investigating geniusApi breadcrumbs...

In [8]:
# extracting genius data...
names = list(gApiData.columns)
genius_data = pd.DataFrame(
    [{"name": str(s), "data": list(gApiData[s].values)[-1]} for s in names]
)
genius_data.dropna(inplace=True)
genius_data["name"] = genius_data["name"].astype(str)
genius_data["data"] = genius_data["data"].apply(lambda x: ast.literal_eval(x))
name_to_genius = dict(zip(genius_data.name, genius_data.data))

In [6]:
# sanity check
name_to_genius["Then the Quiet Explosion"]["url"]

'https://genius.com/Hammock-then-the-quiet-explosion-lyrics'

#### This cell converts a string object into json

In [7]:
# # extracting
# test = list(gApiData["Then the Quiet Explosion"].values)
# test = [val for val in test if val not in [np.nan, 'song', '[]']]

# # jsonifying data...
# parsed_json = ast.literal_eval(test[0])
# parsed_json

### GET Spotify Data...

#### Audio...

In [3]:
# updating audio feature datasets...
helper = spotifyApi()
helper.download_spotify_playlist_audio_features()

Name: b'tide pools ', Number of songs: 4, Playlist ID: 7HCAhke3De2jsnXb4ZFXaf 
Name: b'Unbeknownst Bangers', Number of songs: 27, Playlist ID: 3E684Qjlabk2LmcJjZ2zZQ 
Name: b'sampleLove', Number of songs: 116, Playlist ID: 0DYobfEIwMJlvVBymPoPCf 
Name: b'Costa Rica ', Number of songs: 0, Playlist ID: 46CQNWsKAfp3O2dD12aGV6 
Name: b'purpleParrots pt.3', Number of songs: 34, Playlist ID: 01RLihOalQPRbNAJdCiwOn 
Name: b'Study \xf0\x9f\x93\x9a', Number of songs: 53, Playlist ID: 5c68TrgrsR1COt97GYiHTm 
Name: b'yeye', Number of songs: 12, Playlist ID: 3YUa8BAtVbKmStMYcUnF5z 
Name: b"I'm Just Snacking", Number of songs: 23, Playlist ID: 7f7xecW1muGd5KamjC3EEM 
Name: b'la flame', Number of songs: 472, Playlist ID: 0i3CkxalKp9RRW4jf63PU5 
Name: b'Fashion Killa', Number of songs: 372, Playlist ID: 6k96myNgO28Lt9lZUPqdiU 
Name: b'vamos', Number of songs: 8, Playlist ID: 6pS0A7SrSkuxPSZnsaHd54 
Name: b'View From A Blue Moon', Number of songs: 13, Playlist ID: 0cOa2UTqNsjWO5AJswRZRA 
Name: b'Earga

100%|██████| 14/14 [00:20<00:00,  1.48s/it]


#### Tracklists...

In [4]:
# getting most up-to-date songlist from Spotify...
print("Getting my spotify playlist data...\n")
songs = []
sp = helper.sp
username = helper.username
playlists = helper.get_user_playlists(username, sp)
for playlist in tqdm(playlists):
    songs.extend(helper.get_playlist_content(username, playlist, sp, False))

print(
    f"\nExtracted {len(songs)} songs from {len(playlists)} playlists from your Spotify Account."
)

Getting my spotify playlist data...

Name: b'tide pools ', Number of songs: 4, Playlist ID: 7HCAhke3De2jsnXb4ZFXaf 
Name: b'Unbeknownst Bangers', Number of songs: 27, Playlist ID: 3E684Qjlabk2LmcJjZ2zZQ 
Name: b'sampleLove', Number of songs: 116, Playlist ID: 0DYobfEIwMJlvVBymPoPCf 
Name: b'Costa Rica ', Number of songs: 0, Playlist ID: 46CQNWsKAfp3O2dD12aGV6 
Name: b'purpleParrots pt.3', Number of songs: 34, Playlist ID: 01RLihOalQPRbNAJdCiwOn 
Name: b'Study \xf0\x9f\x93\x9a', Number of songs: 53, Playlist ID: 5c68TrgrsR1COt97GYiHTm 
Name: b'yeye', Number of songs: 12, Playlist ID: 3YUa8BAtVbKmStMYcUnF5z 
Name: b"I'm Just Snacking", Number of songs: 23, Playlist ID: 7f7xecW1muGd5KamjC3EEM 
Name: b'la flame', Number of songs: 472, Playlist ID: 0i3CkxalKp9RRW4jf63PU5 
Name: b'Fashion Killa', Number of songs: 372, Playlist ID: 6k96myNgO28Lt9lZUPqdiU 
Name: b'vamos', Number of songs: 8, Playlist ID: 6pS0A7SrSkuxPSZnsaHd54 
Name: b'View From A Blue Moon', Number of songs: 13, Playlist ID: 

100%|██████| 14/14 [00:08<00:00,  1.71it/s]


Extracted 1333 songs from 14 playlists from your Spotify Account.


In [9]:
# preparing song lists...
sSongs = [val["track"]["name"] for val in songs]
gSongs = [val for val in list(gApiData.columns)]
nSongs = [val for val in songs if val["track"]["name"] not in gSongs]

# sanity checks...
print(len(sSongs), "-", len(gSongs), "=", len(nSongs), "missing songs")

1333 - 999 = 297 missing songs


### GET Genius Lyrics...

#### method 1: web-scraping

In [11]:
url_lyrics = dict()
for songname in tqdm(name_to_genius):
    url = name_to_genius[songname]["url"]
    songlyrics = txtMiner.scrape_genius_lyrics(
        artistname="none", songname=songname, url=url
    )
    url_lyrics[songname] = songlyrics

100%|████| 774/774 [16:32<00:00,  1.28s/it]


In [10]:
# Sanity check
url_lyrics["Then the Quiet Explosion"]

NameError: name 'url_lyrics' is not defined

In [19]:
# saving intermediate results for now...
print("Saving scraped song lyric data... \n")
date_piece = datetime.today().strftime("%Y-%m-%d")
scraped_df = pd.DataFrame([url_lyrics])
scraped_df.to_csv(
    data_path + "scraped_SpotifySongLyrics_" + date_piece + ".csv", index=False
)

Saving scraped song lyric data... 



In [20]:
# Sanity check
count = 0
for key in url_lyrics:
    if len(url_lyrics[key]) < 15:
        count += 1
print(f"Genius web-scraping accuracy: {1 - count/len(url_lyrics.keys())}")

Genius web-scraping accuracy: 0.9573643410852714


In [21]:
# names = [s['track']['name'] for s in nSongs if s['track']['name'] not in list(url_lyrics.keys())]
# [n for n in names if n not in a]

#### method 2: querying Genius API

In [22]:
# checking the the geniusApi for song matches...
print("Querying GENIUS api for track metadata... \n")
song_set = dict()
for song in tqdm(nSongs):
    s = song["track"]
    artistname = s["artists"][0]["name"]
    songname = s["name"]
    songlyrics = txtMiner.search_genius_song(
        track_name=songname, track_artist=artistname
    )
    song_set[songname] = songlyrics

Querying GENIUS api for track metadata... 



  0%|                                       | 0/85 [00:00<?, ?it/s]

Searching for a match for: family ties (with Kendrick Lamar), Baby Keem


  1%|▎                              | 1/85 [00:00<00:35,  2.38it/s]

Searching for a match for: trademark usa, Baby Keem


...

Found a match!
Searching for a match for: COMPLETE MESS, 5 Seconds of Summer


 99%|█████████████████████████████▋| 84/85 [00:39<00:00,  1.52it/s]

Found a match!
Searching for a match for: Head on Fire, Griff


100%|██████████████████████████████| 85/85 [00:40<00:00,  2.12it/s]

Found a match!


In [23]:
# extracting query results...
song_set = dict(
    zip(
        [val for val in song_set if song_set[val]],
        [song_set[val]["result"] for val in song_set if song_set[val]],
    )
)
for songname in tqdm(song_set):
    url = song_set[songname]["url"]
    songlyrics = txtMiner.scrape_genius_lyrics(
        artistname="none", songname=songname, url=url
    )
    url_lyrics[songname] = songlyrics

100%|██████████████████████████████| 66/66 [01:17<00:00,  1.18s/it]


In [24]:
missed = [val for val in nSongs if val["track"]["name"] not in song_set.keys()]
print(f"Genius url accuracy: {(len(nSongs) - len(missed))/len(nSongs)}")

Genius url accuracy: 0.7764705882352941


#### Importing audio datasets...

In [25]:
# importing audio file... (see SpotifySongAudio Wrangling.ipynb for code)
data_path = "/Users/dayoorigunwa/code_base/music_mapping/data/"
allfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
audio_files = [f for f in allfiles if ("12178525311" in f) and (".csv" in f)]

# sanity check
audio_df = pd.DataFrame()
for filename in audio_files:
    tmp_df = pd.read_csv(data_path + filename)
    audio_df = pd.concat([audio_df, tmp_df], axis=0)
audio_df.head()
audio_df["id"] = audio_df["uri"].apply(lambda x: x.split(":")[-1])

In [26]:
# Sanity checks...
count = 0
ids = [s["track"]["id"] for s in songs]
for val in list(audio_df["id"].values):
    if val in ids:
        count += 1
count

889

In [27]:
# extracting genius data...
names = list(gApiData.columns)
genius_data = pd.DataFrame(
    [{"name": str(s), "data": list(gApiData[s].values)[-1]} for s in names]
)
genius_data.dropna(inplace=True)
genius_data["name"] = genius_data["name"].astype(str)
genius_data["data"] = genius_data["data"].apply(lambda x: ast.literal_eval(x))
name_to_genius = dict(zip(genius_data.name, genius_data.data))

In [28]:
# extracting genius data pt2...
url_lyrics = {key: url_lyrics[key] for key in url_lyrics if len(url_lyrics[key]) > 25}
names = list(url_lyrics.keys())
url_lyric_df = pd.DataFrame([{"name": str(s), "lyrics": url_lyrics[s]} for s in names])
url_lyric_df.head()

,name,lyrics
0,Then the Quiet Explosion,I can’t feel you There’s no trace Lights will ...
1,Immunity,You've answered my prayer For a worthless diam...
2,Mumma Don't Tell,My mama don't tell I'm the same My mama don't ...
3,Quick Musical Doodles,[Verse 1] You remember You remember my love Yo...
4,Killing Me To Love You,[Verse 1] Your body is broken but you’re tryin...


In [29]:
# collecting lyric data...
lyric_df_list = [gScrapeLyrics, url_lyric_df]
lyrics_df = pd.DataFrame()
for l_df in lyric_df_list:
    lyrics_df = pd.concat([lyrics_df, l_df], axis=0)
lyrics_dictlist = lyrics_df.to_dict("list")
name_to_lyrics = dict(zip(lyrics_dictlist["name"], lyrics_dictlist["lyrics"]))
lyrics_df.head()

,name,lyrics
0,Then the Quiet Explosion,I can’t feel youThere’s no traceLights will bu...
1,Two Thousand and Seventeen,[Non-Lyrical Vocals]
2,Immunity,You've answered my prayerFor a worthless diamo...
3,Quick Musical Doodles,[Verse 1]You rememberYou remember my loveYou s...
4,Pretending,[Intrumental]


#### Unifying Datasets... ####

In [30]:
# vars...
data = dict()
columns = [
    "name",
    "artist",
    "lyrics",
    "date_added",
    "energy",
    "liveness",
    "tempo",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "time_signature",
    "danceability",
    "valence",
    "duration_ms",
    "loudness",
    "utils_spotify_id",
    "utils_genius_data",
]

for s in songs:

    # get base fields...
    cur_keys = list(data.keys())
    name = s["track"]["name"]
    utils_spotify_id = s["track"]["id"]
    utils_genius_data = name_to_genius.get(name, np.nan)
    artist = s["track"]["artists"][0][
        "name"
    ]  # if s['track']['artists'][0]['name'] else np.nan
    lyrics = name_to_lyrics.get(name, np.nan)
    date_added = s["added_at"]  # if s['added_at'] else np.nan
    audio_row = audio_df[audio_df["id"] == utils_spotify_id]

    # set spotify fields...
    if audio_df[audio_df["id"] == utils_spotify_id].empty == False:
        energy = (
            list(audio_row.energy.values).pop()
            if not audio_row.energy.empty
            else np.nan
        )
        liveness = (
            list(audio_row.liveness.values).pop()
            if not audio_row.liveness.empty
            else np.nan
        )
        tempo = (
            list(audio_row.tempo.values).pop() if not audio_row.tempo.empty else np.nan
        )
        speechiness = (
            list(audio_row.speechiness.values).pop()
            if not audio_row.speechiness.empty
            else np.nan
        )
        acousticness = (
            list(audio_row.acousticness.values).pop()
            if not audio_row.acousticness.empty
            else np.nan
        )
        instrumentalness = (
            list(audio_row.instrumentalness.values).pop()
            if not audio_row.instrumentalness.empty
            else np.nan
        )
        time_signature = (
            list(audio_row.time_signature.values).pop()
            if not audio_row.time_signature.empty
            else np.nan
        )
        danceability = (
            list(audio_row.danceability.values).pop()
            if not audio_row.danceability.empty
            else np.nan
        )
        valence = (
            list(audio_row.valence.values).pop()
            if not audio_row.valence.empty
            else np.nan
        )
        duration_ms = (
            list(audio_row.duration_ms.values).pop()
            if not audio_row.duration_ms.empty
            else np.nan
        )
        loudness = (
            list(audio_row.loudness.values).pop()
            if not audio_row.loudness.empty
            else np.nan
        )
    else:
        energy = np.nan
        liveness = np.nan
        tempo = np.nan
        speechiness = np.nan
        acousticness = np.nan
        instrumentalness = np.nan
        time_signature = np.nan
        danceability = np.nan
        valence = np.nan
        duration_ms = np.nan
        loudness = np.nan

    # wrapping args up...
    values = [
        name,
        artist,
        lyrics,
        date_added,
        energy,
        liveness,
        tempo,
        speechiness,
        acousticness,
        instrumentalness,
        time_signature,
        danceability,
        valence,
        duration_ms,
        loudness,
        utils_spotify_id,
        utils_genius_data,
    ]
    arg_tups = zip(columns, values)

    # build datadict...
    for arg_key, arg_val in list(arg_tups):
        if arg_key in cur_keys:
            data[arg_key].append(arg_val)
        else:
            data[arg_key] = [arg_val]

final_df = pd.DataFrame(columns=columns, data=data)
final_df.head()

,name,artist,lyrics,date_added,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,valence,duration_ms,loudness,utils_spotify_id,utils_genius_data
0,family ties (with Kendrick Lamar),Baby Keem,NaN,2022-04-21T00:18:15Z,0.611,0.230,134.093,0.3300,0.00588,0.000000,4.0,0.711,0.144,252262.0,-5.453,3QFInJAm9eyaho5vBzxInN,NaN
1,trademark usa,Baby Keem,[Part I] [Intro] I can't help but feel neglect...,2022-04-21T00:18:29Z,0.600,0.274,130.732,0.2810,0.10800,0.000002,4.0,0.613,0.067,270671.0,-5.621,6G9aDedv5hYaTgNYDuduqk,NaN
2,VALENTINO,24kGoldn,"[Chorus] I don't want a valentine, I just wa...",2022-04-21T00:19:39Z,0.717,0.132,150.964,0.1790,0.19900,0.000000,4.0,0.746,0.523,179133.0,-4.841,6piAUJJQFD8oHDUr0b7l7q,NaN
3,Pepas,Farruko,"[Letra de ""Pepas""] [Refrán] No me importa lo q...",2022-04-21T00:41:47Z,0.766,0.128,130.001,0.0343,0.00776,0.000070,4.0,0.762,0.442,287120.0,-3.955,5fwSHlTEWpluwOM0Sxnh5k,NaN
4,Stick (with JID & J. Cole feat. Kenny Mason & ...,Dreamville,NaN,2022-04-21T00:42:09Z,0.857,0.668,118.574,0.2920,0.26600,0.000000,4.0,0.671,0.597,309323.0,-5.435,1BzXvBpIFWJgu0P8P6xmP4,NaN


In [31]:
final_df.lyrics.isna().sum()

282

In [32]:
date_piece = datetime.today().strftime("%Y-%m-%d")
final_df.to_csv(data_path + "scraped_dataset_" + date_piece + ".csv")

In [ ]:
final_df.info()